In [1]:
from lxml import etree
from collections import Counter
from io import StringIO, BytesIO
import json
import time
import re
import sys
from sys import getsizeof
from sklearn.feature_extraction.text import CountVectorizer

try:
    import cPickle as pickle
except ImportError:
    import pickle


NAMESPACE = '{http://www.mediawiki.org/xml/export-0.10/}'
PAGE_TAG = NAMESPACE + 'page'
TITLE_TAG = NAMESPACE + 'title'
REDIRECT_TAG = NAMESPACE + 'redirect'
TEXT_TAG = NAMESPACE + 'text'
SOURCE = './wiki-dump/enwiki-1gb'
DEST = './wiki-dump/'


# Fast iteration over xml with low ram consumption
def fast_iter(context, func, dest):
    for event, elem in context:
        # Execute operations over xml node
        func(elem, dest)
        # Clear data read
        elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]
    del context


In [ ]:
# EXTRAER TITULOS NAIVE

start_time = time.time()

context = etree.iterparse('./wiki-dump/enwiki-1gb', events=('end',))

titles = []

for event, elem in context:
    if 'title' in elem.tag:
        if elem.text is not None:
            titles.append(elem.text.lower())
#             titles.append({'title': elem.text.lower(), 'redirect': ''})
#             titles.append((elem.text,))
    elif 'redirect' in elem.tag:
        titles[-1] = elem.attrib['title'].lower()
#         titles[-1]['redirect'] = elem.attrib['title'].lower()
#         titles[-1] = titles[-1] + (elem.attrib['title'],)
    
    elem.clear()
    while elem.getprevious() is not None:
        del elem.getparent()[0]

del context

print('Titles: {} --- {} MB'.format(len(titles), getsizeof(titles) / 1024**2))
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()

with open('./wiki-dump/titles', 'w') as outfile:
    json.dump(titles, outfile, indent=2)
    
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# EXTRAER TITULOS: ALGORITMO ESTRELLA :D

start_time = time.time()

context = etree.iterparse('./wiki-dump/enwiki-1gb', tag=PAGE_TAG , events=('end',))

titles = set()
for event, elem in context:
    try:
        iter_child = elem.iterchildren(tag=REDIRECT_TAG)
        redirect_child = iter_child.__next__()
#         titles.append(redirect_child.attrib['title'].lower())
        titles.add(redirect_child.attrib['title'].lower())

    except StopIteration:
        iter_child = elem.iterchildren(tag=TITLE_TAG)
        title_child = iter_child.__next__()
#         titles.append(title_child.text.lower())
        titles.add(title_child.text.lower())
        
    elem.clear()
    while elem.getprevious() is not None:
        del elem.getparent()[0]

del context


print('Titles: {} --- {} MB'.format(len(titles), getsizeof(titles) / 1024**2))
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
repeated = titles
from collections import Counter
a = dict(Counter(repeated))
print(a)

In [ ]:
count = 1000
for event, elem in etree.iterparse('./wiki-dump/enwiki-1gb', events=('start', 'end')):
    if event == 'start' and 'page' in elem.tag:
        f = elem.find('{http://www.mediawiki.org/xml/export-0.10/}redirect')
        if f is not None:
            print(f.attrib['title'])
        else:
            f = elem.find('{http://www.mediawiki.org/xml/export-0.10/}title')
            print(f.text)
    if count <= 0:
        break
    count -= 1
    elem.clear()
        

In [ ]:
count = 170
text = ''
for event, elem in etree.iterparse('./wiki-dump/enwiki-1gb', events=('start', 'end')):
    if event == 'start' and 'text' in elem.tag:
        count -= 1
        if count == 0:
            text = elem.text
            break

# m = re.findall('\[\[(.*?)]]', text)
m = re.findall('\[\[([^][]+)]]', text)
print(len(m))
print(m)

for i, s in enumerate(m):
    m[i] = s.split('|')[-1]

print(m)


In [ ]:
context1 = etree.iterparse('./wiki-dump/enwiki-1gb', tag=PAGE_TAG , events=('end',))
context2 = etree.iterparse('./wiki-dump/enwiki-1gb', tag=TEXT_TAG , events=('end',))

count1 = 0
for event, elem in context1:
    count1 += 1
    elem.clear()
    while elem.getprevious() is not None:
        del elem.getparent()[0]
    
count2 = 0
for event, elem in context2:
    count2 += 1
    elem.clear()
    while elem.getprevious() is not None:
        del elem.getparent()[0]

del context1  
del context2  
    
print(count1, count2)

In [ ]:
vocabulary = []
alternative_keyphrases = set()

start_time = time.time()

# with open('./wiki-dump/titles', 'wa') as f:

# Extract keyphrases from articles
for event, elem in etree.iterparse('./wiki-dump/enwiki-20160113-pages-articles.xml', events=('start', 'end')):
    title = ''
    # Extract titles of pages
    if event == 'start' and 'title' in elem.tag:
        if elem.text is not None:
#                 vocabulary.append(elem.text.lower())
#                 title = elem.text.lower()
            pass

    elif event == 'start' and 'redirect' in elem.tag:
#             vocabulary[-1] = elem.attrib['title'].lower()
#             title = elem.attrib['title'].lower()
        pass

    # Extract keyphrases in other morphological form
    elif event == 'start' and 'text' in elem.tag:
        text = elem.text
        if text is not None:
#                 keyphrases = re.findall('\[\[([^][]+)]]', text)
#                 for key in keyphrases:
#                     alternative_keyphrases.add(key.split('|')[-1].lower())
            pass

    elem.clear()
        
    
    
print("--- %s seconds ---" % (time.time() - start_time))


start_time = time.time()
vocabulary = alternative_keyphrases.union(set(vocabulary))
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
print('Vocabulary: ', len(vocabulary))
print("--- %s seconds ---" % (time.time() - start_time))

--------------
Split wiki dump xml:
---------------

In [ ]:
from random import random

start_time = time.time()

context = etree.iterparse('./wiki-dump/enwiki-1gb', tag=PAGE_TAG , events=('end',))

xml_header = '<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.10/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.10/ http://www.mediawiki.org/xml/export-0.10.xsd" version="0.10" xml:lang="en">\n'
xml_tail = '</mediawiki>\n'

n = 2
name = './wiki-dump/enwiki-part'

for i in range(1, n + 1):
    filename = name + str(i)
    with open(filename, 'a') as f:
        f.write(xml_header)

# Splits wikipedia dump into n parts choosing randomly the articles of each part
for event, elem in context:
    # Execute operations over xml node
    temp_prob = 0
    rand = random()
    for i in range(1, n + 1):
        if temp_prob <= rand < i / n:
            filename = name + str(i)
            with open(filename, 'ab') as f:
                wiki_page = etree.tostring(elem)
                f.write(wiki_page)
            break
        else:
            temp_prob = i / n
    
    # Clear data read
    elem.clear()
    while elem.getprevious() is not None:
        del elem.getparent()[0]

del context

for i in range(1, n + 1):
    filename = name + str(i)
    with open(filename, 'a') as f:
        f.write(xml_tail)

print("--- %s seconds ---" % (time.time() - start_time))

--------------
Extract keyphrases:
-------------------

In [ ]:
# clean_pattern = '\[\[(image:|file:|category:)(.+)\]\]'
# clean_pattern2 = '\[\[(image:|file:|category:)(.*(<br>\n)*)*]]'
extract_pattern = '\[\[([^][]+)\]\]'

ignore = ['Image:', 'File:', 'Category:', 'Q:', 'image:', 'file:', 'category:', 'q:']

# clean_regex = re.compile(clean_pattern, re.IGNORECASE)
extract_regex = re.compile(extract_pattern, re.IGNORECASE)

context = etree.iterparse('./wiki-dump/enwiki-1gb',
                          tag=PAGE_TAG , events=('end',))

start_time = time.time()

vocabulary = Counter()
# double_key = set()
for event, elem in context:
    new_keys = []
    # Extract titles of pages
    try:
        iterator = elem.iterchildren(tag=REDIRECT_TAG)
        redirect_child = iterator.__next__()
        new_keys.append(redirect_child.attrib['title'].lower())

    except StopIteration:
        iterator = elem.iterchildren(tag=TITLE_TAG)
        title_child = iterator.__next__()
        new_keys.append(title_child.text.lower())

    # Extract keyphrases in other morphological form
    iterator = elem.iterdescendants(tag=TEXT_TAG)
    text_child = iterator.__next__()
    if text_child.text is not None:
#         keyphrases = clean_regex.sub('', text_child.text)
        keyphrases = extract_regex.findall(text_child.text)
        for key in keyphrases:
            if not any(x in key for x in ignore):
                new_keys.append(key.split('|')[-1].lower())
#                 if len(key.split('|')) > 1:
#                     double_key.add(key.lower())
    
    vocabulary.update(new_keys)

    elem.clear()
    while elem.getprevious() is not None:
        del elem.getparent()[0]

del context


print('Vocabulary: {} ngrams --- {} MB'.format(len(vocabulary), getsizeof(vocabulary) / 1024**2))
print("--- %s seconds ---" % (time.time() - start_time))


# Save vocabulary
with open('./wiki-dump/vocabulary_dict', 'wb') as f:
    pickle.dump(vocabulary, f)


In [ ]:
# with open('./wiki-dump/vocabulary_dict', 'rb') as f:
#     voc = pickle.load(f)

print(len(voc.keys()))

In [ ]:
from heapq import nlargest

top_20 = nlargest(20, vocabulary.keys(), key=len)
print([len(i.split()) for i in top_20])
# top_20[50:]
# len("when the pawn hits the conflicts he thinks like a king what he knows throws the blows when he goes to the fight and he'll win the whole thing 'fore he enters the ring there's no body to batter when your mind is your might so when you go solo, you hold your own hand and remember that depth is the greatest of heights and if you know where you stand, then you know where to land and if you fall it won't matter, cuz you'll know that you're right}}".split())
# sum([len(i.split()) for i in vocabulary.keys()]) / len(vocabulary) = 2.5

------------------
Count N-grams in articles
-------------------------

In [ ]:
# from nltk.util import LazyMap

# with open('./wiki-dump/vocabulary', 'rb') as f:
#     voc = pickle.dump(f)

# minivoc = ['titulo ,e', 'otro mas-va', 'simple', '3']

# text = ['simple es el titulo ,e simple 2 y otro mas va',
#         'segunda prueba',
#         'otro mas-va',
#         '2 SIMPLE 3',
#        ]
minivoc = ['màchear', 'no, anda']
text = ['pará @ [[màchear]] no màchear', 
        'màchear màchear no, anda']

# context = etree.iterparse('./wiki-dump/enwiki-1gb', tag=TEXT_TAG , events=('end',))
vec = CountVectorizer(lowercase=True, ngram_range=(1, 10), vocabulary=minivoc, token_pattern='[\S]+')
vec2 = CountVectorizer(lowercase=True, ngram_range=(1, 10), vocabulary=minivoc, token_pattern='\[\[([^][]+)\]\]')
# vec.fit_transform([])

m = vec.transform(text).toarray()
print(vec.get_feature_names())
print(m)

m2 = vec2.transform(text).toarray()
print(vec2.get_feature_names())
print(m2)

re.findall('\[\[([^][]+)\]\]', 'viendo si [[funca|es]] esto')

In [2]:
def get_text(elem):
    text = elem.text
    
    elem.clear()
    while elem.getprevious() is not None:
        del elem.getparent()[0]
        
    return text


context = etree.iterparse('./wiki-dump/enwiki-1gb', tag=TEXT_TAG , events=('end',))

articles = []
count = 1000
for event, elem in context:
    articles.append(get_text(elem))
    
    count -= 1
    if count <= 0:
        break


In [4]:
len(articles)

1000

In [5]:
with open('./wiki-dump/vocabulary_dict', 'rb') as f:
    voc = pickle.load(f)

vec = CountVectorizer(lowercase=True, ngram_range=(1, 20), vocabulary=set(voc.keys()), token_pattern='[\S]+')

# text = [get_text(elem) for event, elem in context]
m = vec.transform(articles)

In [8]:
list(voc.keys())[:10]

['',
 'avenin',
 'glorious twelfth',
 'niigon',
 'herman keiser',
 'huangshan pine',
 'if i ruled the world',
 'generic security services application program interface',
 'hollands kroon',
 "upheld mega bloks' rights to sell their product"]

In [122]:
# s = m.sum(axis=0)
ft_names = vec.get_feature_names()

In [142]:
print(ft_names[956])
voc[""]

!


20

In [113]:
s[0,2]
for i in range(0, 10000):
    if s[0, i] > 0:
        print(s[0, i], i)

1585 956
35 972
1 1000
2 1012
29 1020
2 1026
4 1027
1 1044
9 1082
1 1109
1 1125
2 1158
1 1184
1 1193
1 1207
1 1232
2 1371
2 1493
1 1556
1 1570
5 1604
3 1631
1 1670
4 1762
1 1836
3 1893
1 1917
1 2000
2 2060
2 2096
3 2156
1 2170
1 2196
1 2259
1 2410
1 2510
1 2511
1 2546
1 2670
1 2798
2 2809
1 2835
2 2874
1 3075
1 3207
2 3307
1 3314
1 3338
1 3731
1 3732
1 3738
1 3838
1 3849
2 3909
1 4008
3 4011
471 4024
15 4026
1 4031
4 4032
3 4035
5 4144
19 4145
1 4157
19 4164
1 4168
1 4170
23 4190
1924 4192
169 4211
4 4247
9 4260
2 4271
2 4289
12 4353
7 4369
2 4428
2 4622
6 4644
1 4679
1 4680
1 4739
8 4785
1 4868
1 4899
6 4917
7 4943
1 5104
1 5142
2 5173
2 5214
6 5220
1 5226
2 5287
1 5318
20 5345
1 5368
2 5380
4 5399
2 5403
1 5405
2 5414
4 5441
1 5548
13 5585
1 5626
36 5639
1 5781
20 5884
2 5886
4 5887
5 5894
1 6007
1 6118
27 6181
1 6185
4 6229
1 6339
1 6367
1 6389
2 6470
3 6648
1 6732
3 6745
5 6772
1 6773
2 6810
4 6902
1 6903
1 6949
1 6966
9 7047
1 7062
1 7066
2 7101
13 7143
1 7162
1 7187
1 7255
1 7410